In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0,
    
    groq_api_key = '<your API key here>',
    
    model_name = "llama-3.1-70b-versatile"
    # other params...
)

response = llm.invoke("who is the first person to reach moon!")
print(response.content)

The first person to set foot on the Moon was Neil Armstrong, an American astronaut and the mission commander of the Apollo 11 mission. He stepped onto the lunar surface on July 20, 1969, famously declaring, "That's one small step for man, one giant leap for mankind."

However, it's worth noting that the first person to enter into lunar orbit was Yuri Gagarin's fellow cosmonaut, Alexei Leonov was not the first to walk on the moon but the first to walk in space, but the first person to enter lunar orbit was actually the Soviet Union's Luna 3 spacecraft in 1959, which imaged the far side of the Moon.

The first person to walk on the moon was indeed Neil Armstrong, followed by Edwin "Buzz" Aldrin, who also walked on the Moon during the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-37999")

page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for NIKE Inc. Software Engineering Undergraduate Intern

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsInternshipsNIKE Inc. Software Engineering Undergraduate InternBeaverton, Or

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
{page_data}

### INSTRUCTION:
The scraped text is from the career's page of a website.
Your job is to extract the job postings and return them in JSON format containing
following keys: ` role`, experience, skills and description.
Only return the valid JSON.

### VALID JSON (NO PREAMBLE):

"""
)

In [5]:
chain_extract = prompt_extract | llm

res = chain_extract.invoke(input = {'page_data': page_data})
print(res.content)

```json
{
  "role": "NIKE Inc. Software Engineering Undergraduate Intern",
  "experience": "Previous internship(s) in a digital technology organization, marketing/consumer facing technology organization, operations and logistics team, or enterprise data and analytics team",
  "skills": [
    "Computer programming languages like Java, Python, React, Swift, JavaScript, XCode, C#",
    "Experience working in a cloud-based environment (e.g. Amazon EC2, GCE, Rackspace, Azure, etc.)",
    "Relational and Non-Relational data base technologies like Oracle, MS-SQL, NoSQL, MongoDB, Cassandra",
    "SAP platform and/or Warehouse Management Systems (WMS) like Manhattan"
  ],
  "description": "NIKE, Inc. does more than outfit the world’s best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding imagination. The brand seeks

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
jres = json_parser.parse(res.content)
jres

{'role': 'NIKE Inc. Software Engineering Undergraduate Intern',
 'experience': 'Previous internship(s) in a digital technology organization, marketing/consumer facing technology organization, operations and logistics team, or enterprise data and analytics team',
 'skills': ['Computer programming languages like Java, Python, React, Swift, JavaScript, XCode, C#',
  'Experience working in a cloud-based environment (e.g. Amazon EC2, GCE, Rackspace, Azure, etc.)',
  'Relational and Non-Relational data base technologies like Oracle, MS-SQL, NoSQL, MongoDB, Cassandra',
  'SAP platform and/or Warehouse Management Systems (WMS) like Manhattan'],
 'description': 'NIKE, Inc. does more than outfit the world’s best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding imagination. The brand seeks achievers, leaders and visi

In [7]:
type(jres)

dict

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [10]:
links = collection.query(query_texts= ["Experience in C#"],  n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [11]:
job = jres
job["skills"]

['Computer programming languages like Java, Python, React, Swift, JavaScript, XCode, C#',
 'Experience working in a cloud-based environment (e.g. Amazon EC2, GCE, Rackspace, Azure, etc.)',
 'Relational and Non-Relational data base technologies like Oracle, MS-SQL, NoSQL, MongoDB, Cassandra',
 'SAP platform and/or Warehouse Management Systems (WMS) like Manhattan']

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Vaibhav Kolla, a S/W development Engineer at Reputed Company. Your Company is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of your company 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase company's portfolio: {link_list}
        Remember you are Vaibhav Kolla, SDE at *** Company. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Empowering NIKE Inc.'s Digital Transformation with Customized Software Solutions

Dear Hiring Manager,

I came across the job description for the NIKE Inc. Software Engineering Undergraduate Intern role and was impressed by the company's commitment to innovation and excellence. As a Software Development Engineer at [*** Company], I'd like to introduce our team's capabilities in delivering tailored software solutions that can support NIKE Inc.'s digital transformation goals.

Our company has a proven track record of empowering enterprises with automated tools, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. We've worked with various clients across industries, leveraging our expertise in AI and software consulting to drive business growth.

In particular, our team has extensive experience in developing cloud-based applications, working with relational and non-relational databases, and utilizing programming languages like Java, Pyth